## Predicting 2023 MLB Season: 01 - Get Data + Data Wrangling
This notebook wrangles data downloaded from www.retrosheet.org into a dataframe for model building. Specifically, for each game, calculate team statistics over their past 162 and 30 games.

The resulting dataframe is saved to a file. This file will be the starting point for the next notebook, in which model v1 will be  built.

The game logs in the <raw_data> folder can be found here: https://www.retrosheet.org/gamelogs/index.html


In [1]:
import numpy as np
import pandas as pd

# Display more data
pd.set_option('display.max_columns',1000)
pd.set_option('display.max_rows',1000)

In [2]:
# Testing one of the csvs
fname = './raw_data/gl2000.txt'
df = pd.read_csv(fname, header=None)
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160
0,20000329,0,Wed,CHN,NL,1,NYN,NL,1,5,3,54,N,NaN,NaN,NaN,TOK01,55000,206,100010210,001000020,33,12,1,0,2,5,1,0,1,10,0,5,1,0,4,0,13,3,3,3,0,0,27,10,2,0,1,0,33,7,1,0,1,3,1,1,0,3,0,4,0,0,1,0,8,5,5,5,1,0,27,12,0,0,4,0,marsr901,Randy Marsh,herna901,Angel Hernandez,fostm901,Marty Foster,kulpr901,Ron Kulpa,NaN,(none),NaN,(none),bayld001,Don Baylor,valeb102,Bobby Valentine,liebj001,Jon Lieber,hampm001,Mike Hampton,aguir001,Rick Aguilera,andrs001,Shane Andrews,liebj001,Jon Lieber,hampm001,Mike Hampton,youne001,Eric Young,4,bufod001,Damon Buford,8,gracm001,Mark Grace,3,sosas001,Sammy Sosa,9,rodrh001,Henry Rodriguez,7,andrs001,Shane Andrews,5,nievj002,Jose Nieves,6,giraj001,Joe Girardi,2,liebj001,Jon Lieber,1,hendr001,Rickey Henderson,7,hamid001,Darryl Hamilton,8,alfoe001,Edgardo Alfonzo,4,piazm001,Mike Piazza,2,ventr001,Robin Ventura,5,belld001,Derek Bell,9,zeilt001,Todd Zeile,3,ordor001,Rey Ordonez,6,hampm001,Mike Hampton,1,NaN,Y
1,20000330,0,Thu,NYN,NL,2,CHN,NL,2,5,1,66,N,NaN,NaN,NaN,TOK01,55000,235,00001000004,00001000000,37,6,2,0,1,5,1,1,1,8,0,5,1,0,0,0,10,5,0,0,0,0,33,14,2,0,2,0,36,5,0,0,0,0,2,0,0,6,1,9,0,0,2,0,10,7,5,5,0,0,33,14,0,0,0,0,herna901,Angel Hernandez,fostm901,Marty Foster,kulpr901,Ron Kulpa,marsr901,Randy Marsh,NaN,(none),NaN,(none),valeb102,Bobby Valentine,bayld001,Don Baylor,cookd001,Dennis Cook,yound002,Danny Young,NaN,(none),agbab001,Benny Agbayani,reedr002,Rick Reed,farnk001,Kyle Farnsworth,hendr001,Rickey Henderson,7,hamid001,Darryl Hamilton,8,alfoe001,Edgardo Alfonzo,4,piazm001,Mike Piazza,2,ventr001,Robin Ventura,5,belld001,Derek Bell,9,zeilt001,Todd Zeile,3,ordor001,Rey Ordonez,6,reedr002,Rick Reed,1,youne001,Eric Young,4,bufod001,Damon Buford,8,sosas001,Sammy Sosa,9,gracm001,Mark Grace,3,rodrh001,Henry Rodriguez,7,andrs001,Shane Andrews,5,husoj001,Jeff Huson,6,giraj001,Joe Girardi,2,farnk001,Kyle Farnsworth,1,NaN,Y
2,20000403,0,Mon,COL,NL,1,ATL,NL,1,0,2,51,D,NaN,NaN,NaN,ATL02,42255,134,000000000,00000020x,31,6,2,0,0,0,1,0,0,2,2,7,0,0,1,0,7,3,2,2,1,0,24,10,0,0,1,0,30,7,0,0,2,2,0,0,1,1,0,6,1,0,1,0,6,2,0,0,0,0,27,12,0,0,1,0,hirsj901,John Hirschbeck,willc901,Charlie Williams,wegnm901,Mark Wegner,reynj901,Jim Reynolds,NaN,(none),NaN,(none),bellb001,Buddy Bell,cox-b103,Bobby Cox,maddg002,Greg Maddux,astap001,Pedro Astacio,remlm001,Mike Remlinger,galaa001,Andres Galarraga,astap001,Pedro Astacio,maddg002,Greg Maddux,goodt001,Tom Goodwin,8,lansm001,Mike Lansing,4,walkl001,Larry Walker,9,cirij001,Jeff Cirillo,5,heltt001,Todd Helton,3,hammj001,Jeffrey Hammonds,7,peren001,Neifi Perez,6,maynb001,Brent Mayne,2,astap001,Pedro Astacio,1,veraq001,Quilvio Veras,4,sandr002,Reggie Sanders,7,jonec004,Chipper Jones,5,jordb001,Brian Jordan,9,galaa001,Andres Galarraga,3,jonea002,Andruw Jones,8,peree002,Eddie Perez,2,weisw001,Walt Weiss,6,maddg002,Greg Maddux,1,NaN,Y
3,20000403,0,Mon,MIL,NL,1,CIN,NL,1,3,3,31,D,NaN,NaN,NaN,CIN08,55596,111,002100,21000x,22,7,1,0,0,2,0,0,0,5,0,1,1,0,0,0,8,1,3,3,0,0,15,5,0,0,0,0,19,5,1,0,1,3,0,0,0,1,0,4,0,0,0,0,2,2,2,2,0,0,16,8,2,0,0,0,marsr901,Randy Marsh,herna901,Angel Hernandez,fostm901,Marty Foster,kulpr901,Ron Kulpa,NaN,(none),NaN,(none),loped001,Davey Lopes,mckej801,Jack McKeon,NaN,NaN,NaN,NaN,NaN,(none),NaN,(none),woods001,Steve Woodard,harnp001,Pete Harnisch,grism001,Marquis Grissom,8,lorem001,Mark Loretta,6,burnj001,Jeromy Burnitz,9,jenkg001,Geoff Jenkins,7,hernj001,Jose Hernandez,5,barkk001,Kevin Barker,3,bellr002,Ronnie Belliard,4,blanh001,Henry Blanco,2,woods001,Steve W

In [3]:
# Add column names to df
colnames = ['date','dblheader_code','day_of_week','team_v','league_v','game_no_v',
           'team_h','league_h','game_no_h', 'runs_v', 'runs_h','outs_total','day_night',
            'completion_info','forfeit_info','protest_info','ballpark_id','attendance','game_minutes',
            'linescore_v','linescore_h',
           'AB_v','H_v','2B_v','3B_v','HR_v','RBI_v','SH_v','SF_v','HBP_v','BB_v','IBB_v','SO_v',
            'SB_v', 'CS_v','GIDP_v','CI_v','LOB_v',
            'P_num_v','ERind_v','ERteam_v','WP_v','balk_v',
            'PO_v','ASST_v','ERR_v','PB_v','DP_v','TP_v',
           'AB_h', 'H_h', '2B_h', '3B_h', 'HR_h', 'RBI_h', 'SH_h', 'SF_h', 'HBP_h', 'BB_h', 'IBB_h','SO_h',
            'SB_h', 'CS_h', 'GIDP_h', 'CI_h', 'LOB_h',
            'P_num_h', 'ERind_h', 'ERteam_h', 'WP_h', 'balk_h',
            'PO_h', 'ASST_h', 'ERR_h', 'PB_h', 'DP_h', 'TP_h',
            'ump_HB_id', 'ump_HB_name','ump_1B_id', 'ump_1B_name','ump_2B_id', 'ump_2B_name',
            'ump_3B_id', 'ump_3B_name','ump_LF_id', 'ump_LF_name','ump_RF_id', 'ump_RF_name',
            'mgr_id_v', 'mgr_name_v', 'mgr_id_h', 'mgr_name_h',
            'pitcher_id_w','pitcher_name_w','pitcher_id_l','pitcher_name_l','pitcher_id_s','pitcher_name_s',
            'GWRBI_id','GWRBI_name','pitcher_start_id_v','pitcher_start_name_v','pitcher_start_id_h','pitcher_start_name_h',
            'batter1_name_v', 'batter1_id_v', 'batter1_pos_v', 'batter2_name_v', 'batter2_id_v', 'batter2_pos_v',
            'batter3_name_v', 'batter3_id_v', 'batter3_pos_v', 'batter4_name_v', 'batter4_id_v', 'batter4_pos_v',
            'batter5_name_v', 'batter5_id_v', 'batter5_pos_v', 'batter6_name_v', 'batter6_id_v', 'batter6_pos_v',
            'batter7_name_v', 'batter7_id_v', 'batter7_pos_v', 'batter8_name_v', 'batter8_id_v', 'batter8_pos_v',
            'batter9_name_v', 'batter9_id_v', 'batter9_pos_v', 'batter1_name_h', 'batter1_id_h', 'batter1_pos_h',
            'batter2_name_h', 'batter2_id_h', 'batter2_pos_h', 'batter3_name_h', 'batter3_id_h', 'batter3_pos_h',
            'batter4_name_h', 'batter4_id_h', 'batter4_pos_h', 'batter5_name_h', 'batter5_id_h', 'batter5_pos_h',
            'batter6_name_h', 'batter6_id_h', 'batter6_pos_h', 'batter7_name_h', 'batter7_id_h', 'batter7_pos_h',
            'batter8_name_h', 'batter8_id_h', 'batter8_pos_h', 'batter9_name_h', 'batter9_id_h', 'batter9_pos_h',           
           'misc_info','acqui_info'
           ]

df.columns = colnames
df.sample(10)

,date,dblheader_code,day_of_week,team_v,league_v,game_no_v,team_h,league_h,game_no_h,runs_v,runs_h,outs_total,day_night,completion_info,forfeit_info,protest_info,ballpark_id,attendance,game_minutes,linescore_v,linescore_h,AB_v,H_v,2B_v,3B_v,HR_v,RBI_v,SH_v,SF_v,HBP_v,BB_v,IBB_v,SO_v,SB_v,CS_v,GIDP_v,CI_v,LOB_v,P_num_v,ERind_v,ERteam_v,WP_v,balk_v,PO_v,ASST_v,ERR_v,PB_v,DP_v,TP_v,AB_h,H_h,2B_h,3B_h,HR_h,RBI_h,SH_h,SF_h,HBP_h,BB_h,IBB_h,SO_h,SB_h,CS_h,GIDP_h,CI_h,LOB_h,P_num_h,ERind_h,ERteam_h,WP_h,balk_h,PO_h,ASST_h,ERR_h,PB_h,DP_h,TP_h,ump_HB_id,ump_HB_name,ump_1B_id,ump_1B_name,ump_2B_id,ump_2B_name,ump_3B_id,ump_3B_name,ump_LF_id,ump_LF_name,ump_RF_id,ump_RF_name,mgr_id_v,mgr_name_v,mgr_id_h,mgr_name_h,pitcher_id_w,pitcher_name_w,pitcher_id_l,pitcher_name_l,pitcher_id_s,pitcher_name_s,GWRBI_id,GWRBI_name,pitcher_start_id_v,pitcher_start_name_v,pitcher_start_id_h,pitcher_start_name_h,batter1_name_v,batter1_id_v,batter1_pos_v,batter2_name_v,batter2_id_v,batter2_pos_v,batter3_name_v,batter3_id_v,batter3_pos_v,batter4_name_v,batter4_id_v,batter4_pos_v,batter5_name_v,batter5_id_v,batter5_pos_v,batter6_name_v,batter6_id_v,batter6_pos_v,batter7_name_v,batter7_id_v,batter7_pos_v,batter8_name_v,batter8_id_v,batter8_pos_v,batter9_name_v,batter9_id_v,batter9_pos_v,batter1_name_h,batter1_id_h,batter1_pos_h,batter2_name_h,batter2_id_h,batter2_pos_h,batter3_name_h,batter3_id_h,batter3_pos_h,batter4_name_h,batter4_id_h,batter4_pos_h,batter5_name_h,batter5_id_h,batter5_pos_h,batter6_name_h,batter6_id_h,batter6_pos_h,batter7_name_h,batter7_id_h,batter7_pos_h,batter8_name_h,batter8_id_h,batter8_pos_h,batter9_name_h,batter9_id_h,batter9_pos_h,misc_info,acqui_info
1901,20000825,0,Fri,TBA,AL,126,BAL,AL,127,3,4,51,N,NaN,NaN,NaN,BAL12,34503,200,200000100,00000220x,36,10,2,0,0,3,1,0,0,6,2,7,1,0,0,0,13,4,4,4,0,1,24,10,1,0,1,0,30,7,2,0,1,4,0,0,0,10,0,5,4,0,1,0,12,4,3,3,1,0,27,11,0,0,0,0,kellj901,Jeff Kellogg,diazl901,Laz Diaz,vanvm901,Mike Vanvleet,klemj901,Justin Klemm,NaN,(none),NaN,(none),rothl101,Larry Rothschild,hargm001,Mike Hargrove,ryanb001,B.J. Ryan,taylb001,Billy Taylor,kohlr001,Ryan Kohlmeier,hairj002,Jerry Hairston,lopea002,Albie Lopez,rappp001,Pat Rapp,willg001,Gerald Williams,8,cox-s001,Steve Cox,7,vaugg001,Greg Vaughn,10,mcgrf001,Fred McGriff,3,smitr003,Bob Smith,4,huffa001,Aubrey Huff,5,flahj001,John Flaherty,2,guilj001,Jose Guillen,9,martf001,Felix Martinez,6,andeb001,Brady Anderson,10,moram002,Melvin Mora,6,deshd001,Delino DeShields,7,bellj002,Albert Belle,9,conij001,Jeff Conine,5,richc001,Chris Richard,3,fordb001,Brook Fordyce,2,hairj002,Jerry Hairston,4,matol001,Luis Matos,8,NaN,Y
1617,20000804,0,Fri,TEX,AL,107,TOR,AL,111,8,10,51,N,NaN,NaN,NaN,TOR02,23518,219,021010130,00102052x,35,10,2,0,2,7,1,0,0,4,0,5,0,0,1,0,5,4,8,8,0,0,24,5,2,0,0,0,39,14,3,0,1,9,0,0,2,3,0,3,0,0,0,0,10,7,7,7,2,0,27,10,1,0,1,0,cousd901,Derryl Cousins,marqa901,Alfonso Marquez,brinj901,Joe Brinkman,nelsj901,Jeff Nelson,NaN,(none),NaN,(none),oatej101,Johnny Oates,fregj101,Jim Fregosi,quanp001,Paul Quantrill,venam002,Mike Venafro,kochb001,Billy Koch,fletd001,Darrin Fletcher,perim001,Matt Perisho,tracs001,Steve Trachsel,alicl001,Luis Alicea,4,cataf001,Frank Catalanotto,10,greer001,Rusty Greer,7,palmr001,Rafael Palmeiro,3,kaplg001,Gabe Kapler,8,leder001,Ricky Ledee,9,haseb001,Bill Haselman,2,lambm001,Mike Lamb,5,clayr001,Royce Clayton,6,stews002,Shannon Stewart,7,gonza001,Alex Gonzalez,6,fullb001,Brad Fullmer,10,delgc001,Carlos Delgado,3,batit001,Tony Batista,5,cruzj004,Jose Cruz,8,fletd001,Darrin Fletcher,2,cordm001,Marty Cordova,9,woodc001,Chris Woodward,4,NaN,Y
1829,20000820,0,Sun,MIL,NL,124,HOU,NL,124,6,5,54,D,NaN,NaN,NaN,HOU03,35592,209,001021011,010100300,38,14,4,0,0,5,0,1,1,6,2,12,4,1,0,0,13,3,5,5,0,0,27,6,0,0,0,0,33,7,1,0,2,5,1,0,0,3,2,11,0,0,0,0,5,5,5,5,0,0,27,9,1,0,0,0,nelsj901,Jeff Nelson,marqa901,Alfonso Marquez,fichm901,Mike Fichter,brinj901,Joe Brinkman,NaN,(none),NaN,(none),loped001,Davey Lopes,dierl101,Larry Dierker,leskc001,Curt Leskanic,cabrj001,Jos

## Create a Table with every game since 2000

In [4]:
df = pd.DataFrame()
# Concatenate every file together
for year in range(2000,2024):
    fname = f'./raw_data/gl{year}.txt'
    df_temp = pd.read_csv(fname, header=None)
    #  Assign column names
    df_temp.columns = colnames
    # Add year to df
    df_temp['season'] = year
    df = pd.concat((df, df_temp))
    
# Explore df size
df.shape

(56775, 162)

In [5]:
df.info(max_cols=200)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 56775 entries, 0 to 2429
Data columns (total 162 columns):
 #    Column                Non-Null Count  Dtype  
---   ------                --------------  -----  
 0    date                  56775 non-null  int64  
 1    dblheader_code        56775 non-null  int64  
 2    day_of_week           56775 non-null  object 
 3    team_v                56775 non-null  object 
 4    league_v              56775 non-null  object 
 5    game_no_v             56775 non-null  int64  
 6    team_h                56775 non-null  object 
 7    league_h              56775 non-null  object 
 8    game_no_h             56775 non-null  int64  
 9    runs_v                56775 non-null  int64  
 10   runs_h                56775 non-null  int64  
 11   outs_total            56775 non-null  int64  
 12   day_night             56775 non-null  object 
 13   completion_info       49 non-null     object 
 14   forfeit_info          0 non-null      float64
 15   p

In [6]:
## Calculate additional useful columns
df['run_diff'] = df['runs_h']-df['runs_v']
df['home_victory'] = (df['run_diff']>0).astype(int)
df['run_total'] = df['runs_h'].copy()+df['runs_v'].copy()
df['date_dblhead'] = (df['date'].astype(str) + df['dblheader_code'].astype(str)).astype(int)

In [7]:
# On average, how often did the home team win?
df.home_victory.mean()

0.5380713342140027

In [8]:
# Retrieve single teams games
df_yankees = df.loc[((df.team_v=='NYA') | (df.team_h=='NYA'))]
df_yankees.shape

(3784, 166)

In [9]:
df_yankees.head(100)

,date,dblheader_code,day_of_week,team_v,league_v,game_no_v,team_h,league_h,game_no_h,runs_v,runs_h,outs_total,day_night,completion_info,forfeit_info,protest_info,ballpark_id,attendance,game_minutes,linescore_v,linescore_h,AB_v,H_v,2B_v,3B_v,HR_v,RBI_v,SH_v,SF_v,HBP_v,BB_v,IBB_v,SO_v,SB_v,CS_v,GIDP_v,CI_v,LOB_v,P_num_v,ERind_v,ERteam_v,WP_v,balk_v,PO_v,ASST_v,ERR_v,PB_v,DP_v,TP_v,AB_h,H_h,2B_h,3B_h,HR_h,RBI_h,SH_h,SF_h,HBP_h,BB_h,IBB_h,SO_h,SB_h,CS_h,GIDP_h,CI_h,LOB_h,P_num_h,ERind_h,ERteam_h,WP_h,balk_h,PO_h,ASST_h,ERR_h,PB_h,DP_h,TP_h,ump_HB_id,ump_HB_name,ump_1B_id,ump_1B_name,ump_2B_id,ump_2B_name,ump_3B_id,ump_3B_name,ump_LF_id,ump_LF_name,ump_RF_id,ump_RF_name,mgr_id_v,mgr_name_v,mgr_id_h,mgr_name_h,pitcher_id_w,pitcher_name_w,pitcher_id_l,pitcher_name_l,pitcher_id_s,pitcher_name_s,GWRBI_id,GWRBI_name,pitcher_start_id_v,pitcher_start_name_v,pitcher_start_id_h,pitcher_start_name_h,batter1_name_v,batter1_id_v,batter1_pos_v,batter2_name_v,batter2_id_v,batter2_pos_v,batter3_name_v,batter3_id_v,batter3_pos_v,batter4_name_v,batter4_id_v,batter4_pos_v,batter5_name_v,batter5_id_v,batter5_pos_v,batter6_name_v,batter6_id_v,batter6_pos_v,batter7_name_v,batter7_id_v,batter7_pos_v,batter8_name_v,batter8_id_v,batter8_pos_v,batter9_name_v,batter9_id_v,batter9_pos_v,batter1_name_h,batter1_id_h,batter1_pos_h,batter2_name_h,batter2_id_h,batter2_pos_h,batter3_name_h,batter3_id_h,batter3_pos_h,batter4_name_h,batter4_id_h,batter4_pos_h,batter5_name_h,batter5_id_h,batter5_pos_h,batter6_name_h,batter6_id_h,batter6_pos_h,batter7_name_h,batter7_id_h,batter7_pos_h,batter8_name_h,batter8_id_h,batter8_pos_h,batter9_name_h,batter9_id_h,batter9_pos_h,misc_info,acqui_info,season,run_diff,home_victory,run_total,date_dblhead
8,20000403,0,Mon,NYA,AL,1,ANA,AL,1,3,2,54,N,NaN,NaN,NaN,ANA01,42704.0,182,000002100,010000001,32,6,0,0,2,3,0,0,0,3,0,3,0,2,0,0,5,3,2,2,0,0,27,5,0,0,1,0,35,10,1,0,1,2,0,0,0,5,0,6,0,1,0,0,11,3,3,3,0,0,27,16,1,0,0,0,mcclt901,Tim McClelland,craft901,Terry Craft,schrp901,Paul Schrieber,cuzzp901,Phil Cuzzi,NaN,(none),NaN,(none),torrj101,Joe Torre,sciom001,Mike Scioscia,herno001,Orlando Hernandez,hillk001,Ken Hill,rivem002,Mariano Rivera,oneip001,Paul O'Neill,herno001,Orlando Hernandez,hillk001,Ken Hill,knobc001,Chuck Knoblauch,4,jeted001,Derek Jeter,6,oneip001,Paul O'Neill,9,willb002,Bernie Williams,10,martt002,Tino Martinez,3,leder001,Ricky Ledee,8,posaj001,Jorge Posada,2,spens001,Shane Spencer,7,bross001,Scott Brosius,5,erstd001,Darin Erstad,7,kenna001,Adam Kennedy,4,vaugm001,Mo Vaughn,3,salmt001,Tim Salmon,9,andeg001,Garret Anderson,8,glaut001,Troy Glaus,5,spies001,Scott Spiezio,10,molib001,Bengie Molina,2,disag001,Gary Disarcina,6,NaN,Y,2000,-1,0,5,200004030
20,20000404,0,Tue,NYA,AL,2,ANA,AL,2,5,3,54,N,NaN,NaN,NaN,ANA01,25818.0,200,000101102,000003000,35,9,2,1,1,5,0,0,0,6,2,7,1,0,2,0,9,4,0,0,0,0,27,11,1,0,2,0,36,10,0,0,0,3,0,0,0,6,0,9,1,0,2,0,12,4,5,5,0,0,27,12,1,0,2,0,craft901,Terry Craft,schrp901,Paul Schrieber,cuzzp901,Phil Cuzzi,mcclt901,Tim McClelland,NaN,(none),NaN,(none),torrj101,Joe Torre,sciom001,Mike Scioscia,mendr001,Ramiro Mendoza,perct001,Troy Percival,rivem002,Mariano Rivera,willb002,Bernie Williams,clemr001,Roger Clemens,bottk001,Kent Bottenfield,knobc001,Chuck Knoblauch,4,jeted001,Derek Jeter,6,oneip001,Paul O'Neill,9,willb002,Bernie Williams,10,martt002,Tino Martinez,3,leder001,Ricky Ledee,8,posaj001,Jorge Posada,2,spens001,Shane Spencer,7,bellc001,Clay Bellinger,5,erstd001,Darin Erstad,7,kenna001,Adam Kennedy,4,vaugm001,Mo Vaughn,3,salmt001,Tim Salmon,9,andeg001,Garret Anderson,8,glaut001,Troy Glaus,5,spies001,Scott Spiezio,10,molib001,Bengie Molina,2,disag001,Gary Disarcina,6,NaN,Y,2000,-2,0,8,200004040
34,20000405,0,Wed,NYA,AL,3,ANA,AL,3,6,12,51,N,NaN,NaN,NaN,ANA01,24560.0,181,001202010,12610110x,40,13,3,0,2,6,0,0,0,0,0,5,1,0,2,0,7,4,12,12,0,0,24,8,0,0,0,0,33,12,4,0,1,12,0,2,1,8,0,4,1,0,0,0,8,3,6,6,0,0,27,13,2,0,2,0,schrp901,Paul Schrieber,cuzzp901,Phil Cuzzi,mcclt901,Tim McClelland,craft901,Terry Craft,NaN,(none),N

In [10]:
## Create a team-specific data frame, given the team

# Remove home or away suffix
def strip_suffix(x, suff):
    if x.endswith(suff):
        return(x[:-len(suff)])
    else:
        return(x)

# Remove home columns for visitor table
visit_cols = [col for col in df.columns if not col.endswith('_h')]
# Strip the suffix from remaining columns
visit_cols_stripped = [strip_suffix(col,'_v') for col in visit_cols]

home_cols = [col for col in df.columns if not col.endswith('_v')]
home_cols_stripped = [strip_suffix(col,'_h') for col in home_cols]

## This subsets the game level df by team, to aggregate team statistics easily
## Create rolling sums with an offset, so that the rollsum number represents statistics up to, but not including, the game in question

def create_team_df(team):
    df_team_v = df[(df.team_v==team)]
    opponent = df_team_v['team_h']
    df_team_v = df_team_v[visit_cols]
    df_team_v.columns = visit_cols_stripped
    df_team_v['home_game'] = 0
    df_team_v['opponent'] = opponent

    df_team_h = df[(df.team_h==team)]
    opponent = df_team_h['team_v']
    df_team_h = df_team_h[home_cols]
    df_team_h.columns = home_cols_stripped
    df_team_h['home_game'] = 1
    df_team_h['opponent'] = opponent


    df_team = pd.concat((df_team_h, df_team_v))
    df_team.sort_values(['date', 'game_no'],inplace=True)

    # Creating rollsums for the previous 162 and 30 games
    for winsize in [162,30]:
        suff = str(winsize)
        # Create rolloing sum for all basic metrics
        for raw_col in ['AB','H','2B','3B','HR','BB','runs','SO','SB','CS','ERR']:
            new_col = f'rollsum_{raw_col}_{suff}'
            df_team[new_col] = df_team[raw_col].rolling(winsize, closed='left').sum()

        df_team[f'rollsum_BATAVG_{suff}'] = df_team[f'rollsum_H_{suff}'] / df_team[f'rollsum_AB_{suff}']
        df_team[f'rollsum_OBP_{suff}'] = (df_team[f'rollsum_H_{suff}'] + df_team[f'rollsum_BB_{suff}']) / (
                                    df_team[f'rollsum_AB_{suff}']+df_team[f'rollsum_BB_{suff}'])
        df_team[f'rollsum_SLG_{suff}'] = (df_team[f'rollsum_H_{suff}'] + df_team[f'rollsum_2B_{suff}'] 
                                 + 2*df_team[f'rollsum_3B_{suff}']+ 
                                3*df_team[f'rollsum_HR_{suff}'] ) / (df_team[f'rollsum_AB_{suff}'])
        df_team[f'rollsum_OPS_{suff}'] = df_team[f'rollsum_OBP_{suff}'] + df_team[f'rollsum_SLG_{suff}']
        df_team[f'rollsum_SO_perc_{suff}'] = df_team[f'rollsum_SO_{suff}'] / df_team[f'rollsum_AB_{suff}']
        df_team[f'rollsum_BB_perc_{suff}'] = df_team[f'rollsum_BB_{suff}'] / df_team[f'rollsum_AB_{suff}']

    df_team['season_game'] = df_team['season']*1000 + df_team['game_no']
    df_team.set_index('season_game', inplace=True)
    return(df_team)

In [11]:
df_yankees = create_team_df('NYA')
df_yankees.sample(10)

,date,dblheader_code,day_of_week,team,league,game_no,runs,outs_total,day_night,completion_info,forfeit_info,protest_info,ballpark_id,attendance,game_minutes,linescore,AB,H,2B,3B,HR,RBI,SH,SF,HBP,BB,IBB,SO,SB,CS,GIDP,CI,LOB,P_num,ERind,ERteam,WP,balk,PO,ASST,ERR,PB,DP,TP,ump_HB_id,ump_HB_name,ump_1B_id,ump_1B_name,ump_2B_id,ump_2B_name,ump_3B_id,ump_3B_name,ump_LF_id,ump_LF_name,ump_RF_id,ump_RF_name,mgr_id,mgr_name,pitcher_id_w,pitcher_name_w,pitcher_id_l,pitcher_name_l,pitcher_id_s,pitcher_name_s,GWRBI_id,GWRBI_name,pitcher_start_id,pitcher_start_name,batter1_name,batter1_id,batter1_pos,batter2_name,batter2_id,batter2_pos,batter3_name,batter3_id,batter3_pos,batter4_name,batter4_id,batter4_pos,batter5_name,batter5_id,batter5_pos,batter6_name,batter6_id,batter6_pos,batter7_name,batter7_id,batter7_pos,batter8_name,batter8_id,batter8_pos,batter9_name,batter9_id,batter9_pos,misc_info,acqui_info,season,run_diff,home_victory,run_total,date_dblhead,home_game,opponent,rollsum_AB_162,rollsum_H_162,rollsum_2B_162,rollsum_3B_162,rollsum_HR_162,rollsum_BB_162,rollsum_runs_162,rollsum_SO_162,rollsum_SB_162,rollsum_CS_162,rollsum_ERR_162,rollsum_BATAVG_162,rollsum_OBP_162,rollsum_SLG_162,rollsum_OPS_162,rollsum_SO_perc_162,rollsum_BB_perc_162,rollsum_AB_30,rollsum_H_30,rollsum_2B_30,rollsum_3B_30,rollsum_HR_30,rollsum_BB_30,rollsum_runs_30,rollsum_SO_30,rollsum_SB_30,rollsum_CS_30,rollsum_ERR_30,rollsum_BATAVG_30,rollsum_OBP_30,rollsum_SLG_30,rollsum_OPS_30,rollsum_SO_perc_30,rollsum_BB_perc_30
season_game,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2007162,20070930,0,Sun,NYA,AL,162,10,54,D,NaN,NaN,NaN,BAL12,43589.0,206,101430100,40,14,2,1,0,10,0,1,1,8,0,10,1,0,0,0,13,6,4,4,0,0,27,11,0,1,1,0,randt901,Tony Randazzo,gibsg901,Greg Gibson,vanol901,Larry Vanover,demud901,Dana DeMuth,NaN,(none),NaN,(none),torrj101,Joe Torre,wrigc001,Chase Wright,burrb001,Brian Burres,NaN,(none),betew001,Wilson Betemit,henns001,Sean Henn,damoj001,Johnny Damon,7,jeted001,Derek Jeter,6,abreb001,Bobby Abreu,9,rodra001,Alex Rodriguez,5,giamj001,Jason Giambi,10,betew001,Wilson Betemit,4,cabrm002,Melky Cabrera,8,miend001,Doug Mientkiewicz,3,molij001,Jose Molina,2,NaN,Y,2007,-6,0,14,200709300,0,BAL,5715.0,1653.0,328.0,31.0,202.0,633.0,963.0,992.0,127.0,40.0,89.0,0.289239,0.360113,0.463517,0.823630,0.173578,0.110761,1067.0,301.0,68.0,5.0,37.0,126.0,183.0,202.0,23.0,10.0,18.0,0.282099,0.357921,0.459231,0.817153,0.189316,0.118088
2014029,20140503,0,Sat,NYA,AL,29,9,51,D,NaN,NaN,NaN,NYC21,43325.0,211,00021123x,35,12,4,0,2,9,0,1,0,4,1,6,1,0,0,0,7,3,3,3,0,0,27,11,0,0,0,0,eddid901,Doug Eddings,hudsm901,Marvin Hudson,blasc901,Cory Blaser,onorb901,Brian O'Nora,NaN,(none),NaN,(none),giraj001,Joe Girardi,tanam001,Masahiro Tanaka,luekj001,Josh Lueke,NaN,(none),johnk003,Kelly Johnson,tanam001,Masahiro Tanaka,ellsj001,Jacoby Ellsbury,8,gardb001,Brett Gardner,7,teixm001,Mark Teixeira,3,mccab002,Brian McCann,2,soria001,Alfonso Soriano,10,johnk003,Kelly Johnson,5,robeb003,Brian Roberts,4,suzui001,Ichiro Suzuki,9,solay001,Yangervis Solarte,6,NaN,Y,2014,6,1,12,201405030,1,TBA,5492.0,1335.0,264.0,23.0,132.0,459.0,642.0,1244.0,125.0,30.0,70.0,0.243081,0.301462,0.371631,0.673093,0.226511,0.083576,1074.0,279.0,60.0,5.0,27.0,85.0,124.0,239.0,23.0,4.0,17.0,0.259777,0.314064,0.400372,0.714436,0.222533,0.079143
2001151,20010926,0,Wed,NYA,AL,151,5,51,N,NaN,NaN,NaN,NYC16,23352.0,175,02010002x,35,8,3,0,1,5,0,0,0,1,0,7,0,0,0,0,7,4,1,1,0,0,27,8,0,0,2,0,bellw901,Wally Bell,fostm901,Marty Foster,hirsm901,Mark Hirschbeck,kulpr901,Ron Kulpa,NaN,(none),NaN,(none),torrj101,Joe Torre,lillt001,Ted Lilly,wilsp001,Paul Wilson,NaN,(none),bross001,Scott Brosius,petta001,Andy Pettitte,soria001,Alfonso Soriano,4,johnn001,Nick Johnson,10,jeted001,Derek Jeter,6,willb002,Bernie Williams,8,martt002,Tino Martinez,3,spens001,Shane Spencer,9,bross001,Scott Brosius,5,greet003,Todd Greene,2,bellc001,Clay Bellinger,7,NaN,Y,2001,4,1,6,

In [12]:
# Create the team level dataframe for each team - put in dict for easy access
team_data_dict = {}
for team in df.team_v.unique():
    team_data_dict[team] = create_team_df(team)

team_data_dict['NYA']

,date,dblheader_code,day_of_week,team,league,game_no,runs,outs_total,day_night,completion_info,forfeit_info,protest_info,ballpark_id,attendance,game_minutes,linescore,AB,H,2B,3B,HR,RBI,SH,SF,HBP,BB,IBB,SO,SB,CS,GIDP,CI,LOB,P_num,ERind,ERteam,WP,balk,PO,ASST,ERR,PB,DP,TP,ump_HB_id,ump_HB_name,ump_1B_id,ump_1B_name,ump_2B_id,ump_2B_name,ump_3B_id,ump_3B_name,ump_LF_id,ump_LF_name,ump_RF_id,ump_RF_name,mgr_id,mgr_name,pitcher_id_w,pitcher_name_w,pitcher_id_l,pitcher_name_l,pitcher_id_s,pitcher_name_s,GWRBI_id,GWRBI_name,pitcher_start_id,pitcher_start_name,batter1_name,batter1_id,batter1_pos,batter2_name,batter2_id,batter2_pos,batter3_name,batter3_id,batter3_pos,batter4_name,batter4_id,batter4_pos,batter5_name,batter5_id,batter5_pos,batter6_name,batter6_id,batter6_pos,batter7_name,batter7_id,batter7_pos,batter8_name,batter8_id,batter8_pos,batter9_name,batter9_id,batter9_pos,misc_info,acqui_info,season,run_diff,home_victory,run_total,date_dblhead,home_game,opponent,rollsum_AB_162,rollsum_H_162,rollsum_2B_162,rollsum_3B_162,rollsum_HR_162,rollsum_BB_162,rollsum_runs_162,rollsum_SO_162,rollsum_SB_162,rollsum_CS_162,rollsum_ERR_162,rollsum_BATAVG_162,rollsum_OBP_162,rollsum_SLG_162,rollsum_OPS_162,rollsum_SO_perc_162,rollsum_BB_perc_162,rollsum_AB_30,rollsum_H_30,rollsum_2B_30,rollsum_3B_30,rollsum_HR_30,rollsum_BB_30,rollsum_runs_30,rollsum_SO_30,rollsum_SB_30,rollsum_CS_30,rollsum_ERR_30,rollsum_BATAVG_30,rollsum_OBP_30,rollsum_SLG_30,rollsum_OPS_30,rollsum_SO_perc_30,rollsum_BB_perc_30
season_game,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2000001,20000403,0,Mon,NYA,AL,1,3,54,N,NaN,NaN,NaN,ANA01,42704.0,182,000002100,32,6,0,0,2,3,0,0,0,3,0,3,0,2,0,0,5,3,2,2,0,0,27,5,0,0,1,0,mcclt901,Tim McClelland,craft901,Terry Craft,schrp901,Paul Schrieber,cuzzp901,Phil Cuzzi,NaN,(none),NaN,(none),torrj101,Joe Torre,herno001,Orlando Hernandez,hillk001,Ken Hill,rivem002,Mariano Rivera,oneip001,Paul O'Neill,herno001,Orlando Hernandez,knobc001,Chuck Knoblauch,4,jeted001,Derek Jeter,6,oneip001,Paul O'Neill,9,willb002,Bernie Williams,10,martt002,Tino Martinez,3,leder001,Ricky Ledee,8,posaj001,Jorge Posada,2,spens001,Shane Spencer,7,bross001,Scott Brosius,5,NaN,Y,2000,-1,0,5,200004030,0,ANA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000002,20000404,0,Tue,NYA,AL,2,5,54,N,NaN,NaN,NaN,ANA01,25818.0,200,000101102,35,9,2,1,1,5,0,0,0,6,2,7,1,0,2,0,9,4,0,0,0,0,27,11,1,0,2,0,craft901,Terry Craft,schrp901,Paul Schrieber,cuzzp901,Phil Cuzzi,mcclt901,Tim McClelland,NaN,(none),NaN,(none),torrj101,Joe Torre,mendr001,Ramiro Mendoza,perct001,Troy Percival,rivem002,Mariano Rivera,willb002,Bernie Williams,clemr001,Roger Clemens,knobc001,Chuck Knoblauch,4,jeted001,Derek Jeter,6,oneip001,Paul O'Neill,9,willb002,Bernie Williams,10,martt002,Tino Martinez,3,leder001,Ricky Ledee,8,posaj001,Jorge Posada,2,spens001,Shane Spencer,7,bellc001,Clay Bellinger,5,NaN,Y,2000,-2,0,8,200004040,0,ANA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000003,20000405,0,Wed,NYA,AL,3,6,51,N,NaN,NaN,NaN,ANA01,24560.0,181,001202010,40,13,3,0,2,6,0,0,0,0,0,5,1,0,2,0,7,4,12,12,0,0,24,8,0,0,0,0,schrp901,Paul Schrieber,cuzzp901,Phil Cuzzi,mcclt901,Tim McClelland,craft901,Terry Craft,NaN,(none),NaN,(none),torrj101,Joe Torre,schos001,Scott Schoeneweis,coned001,David Cone,NaN,(none),erstd001,Darin Erstad,coned001,David Cone,knobc001,Chuck Knoblauch,4,jeted001,Derek Jeter,6,oneip001,Paul O'Neill,9,willb002,Bernie Williams,10,spens001,Shane Spencer,7,martt002,Tino Martinez,3,posaj001,Jorge Posada,2,kellb002,Roberto Kelly,8,bellc001,Clay Bellinger,5,NaN,Y,2000,6,1,18,200004050,0,ANA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000004,20000407

In [13]:
## Create a variety of summarized statistics for each game
## For each game, we look up the home and visiting team in the team data dictionary, and then look up the game, and pull the relevant stats

BATAVG_162_h = np.zeros(df.shape[0])
BATAVG_162_v = np.zeros(df.shape[0])
OBP_162_h = np.zeros(df.shape[0])
OBP_162_v = np.zeros(df.shape[0])
SLG_162_h = np.zeros(df.shape[0])
SLG_162_v = np.zeros(df.shape[0])
OPS_162_h = np.zeros(df.shape[0])
OPS_162_v = np.zeros(df.shape[0])
SB_162_h = np.zeros(df.shape[0])
SB_162_v = np.zeros(df.shape[0])
CS_162_h = np.zeros(df.shape[0])
CS_162_v = np.zeros(df.shape[0])
ERR_162_h = np.zeros(df.shape[0])
ERR_162_v = np.zeros(df.shape[0])
SO_perc_162_h = np.zeros(df.shape[0])
SO_perc_162_v = np.zeros(df.shape[0])
BB_perc_162_h = np.zeros(df.shape[0])
BB_perc_162_v = np.zeros(df.shape[0])
BATAVG_30_h = np.zeros(df.shape[0])
BATAVG_30_v = np.zeros(df.shape[0])
OBP_30_h = np.zeros(df.shape[0])
OBP_30_v = np.zeros(df.shape[0])
SLG_30_h = np.zeros(df.shape[0])
SLG_30_v = np.zeros(df.shape[0])
OPS_30_h = np.zeros(df.shape[0])
OPS_30_v = np.zeros(df.shape[0])
SB_30_h = np.zeros(df.shape[0])
SB_30_v = np.zeros(df.shape[0])
CS_30_h = np.zeros(df.shape[0])
CS_30_v = np.zeros(df.shape[0])
ERR_30_h = np.zeros(df.shape[0])
ERR_30_v = np.zeros(df.shape[0])
SO_perc_30_h = np.zeros(df.shape[0])
SO_perc_30_v = np.zeros(df.shape[0])
BB_perc_30_h = np.zeros(df.shape[0])
BB_perc_30_v = np.zeros(df.shape[0])


i=0
for index, row in df.iterrows():
    if i%1000==0:
        print(i)
    home_team = row['team_h']
    visit_team = row['team_v']
    game_index_v = row['season']*1000 + row['game_no_v']
    game_index_h = row['season']*1000 + row['game_no_h']
    BATAVG_162_h[i] = team_data_dict[home_team].loc[game_index_h,'rollsum_BATAVG_162']
    BATAVG_162_v[i] = team_data_dict[visit_team].loc[game_index_v,'rollsum_BATAVG_162']
    OBP_162_h[i] = team_data_dict[home_team].loc[game_index_h,'rollsum_OBP_162']
    OBP_162_v[i] = team_data_dict[visit_team].loc[game_index_v,'rollsum_OBP_162']
    SLG_162_h[i] = team_data_dict[home_team].loc[game_index_h,'rollsum_SLG_162']
    SLG_162_v[i] = team_data_dict[visit_team].loc[game_index_v,'rollsum_SLG_162']
    OPS_162_h[i] = team_data_dict[home_team].loc[game_index_h,'rollsum_OPS_162']
    OPS_162_v[i] = team_data_dict[visit_team].loc[game_index_v,'rollsum_OPS_162']
    SB_162_h[i] = team_data_dict[home_team].loc[game_index_h,'rollsum_SB_162']
    SB_162_v[i] = team_data_dict[visit_team].loc[game_index_v,'rollsum_SB_162']
    CS_162_h[i] = team_data_dict[home_team].loc[game_index_h,'rollsum_CS_162']
    CS_162_v[i] = team_data_dict[visit_team].loc[game_index_v,'rollsum_CS_162']
    ERR_162_h[i] = team_data_dict[home_team].loc[game_index_h,'rollsum_ERR_162']
    ERR_162_v[i] = team_data_dict[visit_team].loc[game_index_v,'rollsum_ERR_162']
    SO_perc_162_h[i] = team_data_dict[home_team].loc[game_index_h,'rollsum_SO_perc_162']
    SO_perc_162_v[i] = team_data_dict[visit_team].loc[game_index_v,'rollsum_SO_perc_162']
    BB_perc_162_h[i] = team_data_dict[home_team].loc[game_index_h,'rollsum_BB_perc_162']
    BB_perc_162_v[i] = team_data_dict[visit_team].loc[game_index_v,'rollsum_BB_perc_162']
    BATAVG_30_h[i] = team_data_dict[home_team].loc[game_index_h,'rollsum_BATAVG_30']
    BATAVG_30_v[i] = team_data_dict[visit_team].loc[game_index_v,'rollsum_BATAVG_30']
    OBP_30_h[i] = team_data_dict[home_team].loc[game_index_h,'rollsum_OBP_30']
    OBP_30_v[i] = team_data_dict[visit_team].loc[game_index_v,'rollsum_OBP_30']
    SLG_30_h[i] = team_data_dict[home_team].loc[game_index_h,'rollsum_SLG_30']
    SLG_30_v[i] = team_data_dict[visit_team].loc[game_index_v,'rollsum_SLG_30']
    OPS_30_h[i] = team_data_dict[home_team].loc[game_index_h,'rollsum_OPS_30']
    OPS_30_v[i] = team_data_dict[visit_team].loc[game_index_v,'rollsum_OPS_30']
    SB_30_h[i] = team_data_dict[home_team].loc[game_index_h,'rollsum_SB_30']
    SB_30_v[i] = team_data_dict[visit_team].loc[game_index_v,'rollsum_SB_30']
    CS_30_h[i] = team_data_dict[home_team].loc[game_index_h,'rollsum_CS_30']
    CS_30_v[i] = team_data_dict[visit_team].loc[game_index_v,'rollsum_CS_30']
    ERR_30_h[i] = team_data_dict[home_team].loc[game_index_h,'rollsum_ERR_30']
    ERR_30_v[i] = team_data_dict[visit_team].loc[game_index_v,'rollsum_ERR_30']
    SO_perc_30_h[i] = team_data_dict[home_team].loc[game_index_h,'rollsum_SO_perc_30']
    SO_perc_30_v[i] = team_data_dict[visit_team].loc[game_index_v,'rollsum_SO_perc_30']
    BB_perc_30_h[i] = team_data_dict[home_team].loc[game_index_h,'rollsum_BB_perc_30']
    BB_perc_30_v[i] = team_data_dict[visit_team].loc[game_index_v,'rollsum_BB_perc_30']
    i+=1

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000


In [14]:
# Add constructed arrays into the main game level dataframe
df['BATAVG_162_h'] = BATAVG_162_h
df['BATAVG_162_v'] = BATAVG_162_v
df['OBP_162_h'] = OBP_162_h
df['OBP_162_v'] = OBP_162_v
df['SLG_162_h'] = SLG_162_h
df['SLG_162_v'] = SLG_162_v
df['OPS_162_h'] = OPS_162_h
df['OPS_162_v'] = OPS_162_v
df['SB_162_h'] = SB_162_h
df['SB_162_v'] = SB_162_v
df['CS_162_h'] = CS_162_h
df['CS_162_v'] = CS_162_v
df['ERR_162_h'] = ERR_162_h
df['ERR_162_v'] = ERR_162_v
df['SO_perc_162_h'] = SO_perc_162_h
df['SO_perc_162_v'] = SO_perc_162_v
df['BB_perc_162_h'] = BB_perc_162_h
df['BB_perc_162_v'] = BB_perc_162_v
df['BATAVG_30_h'] = BATAVG_30_h
df['BATAVG_30_v'] = BATAVG_30_v
df['OBP_30_h'] = OBP_30_h
df['OBP_30_v'] = OBP_30_v
df['SLG_30_h'] = SLG_30_h
df['SLG_30_v'] = SLG_30_v
df['OPS_30_h'] = OPS_30_h
df['OPS_30_v'] = OPS_30_v
df['SB_30_h'] = SB_30_h
df['SB_30_v'] = SB_30_v
df['CS_30_h'] = CS_30_h
df['CS_30_v'] = CS_30_v
df['ERR_30_h'] = ERR_30_h
df['ERR_30_v'] = ERR_30_v
df['SO_perc_30_h'] = SO_perc_30_h
df['SO_perc_30_v'] = SO_perc_30_v
df['BB_perc_30_h'] = BB_perc_30_h
df['BB_perc_30_v'] = BB_perc_30_v

In [15]:
df.shape

(56775, 202)

In [16]:
df.sample(5)

,date,dblheader_code,day_of_week,team_v,league_v,game_no_v,team_h,league_h,game_no_h,runs_v,runs_h,outs_total,day_night,completion_info,forfeit_info,protest_info,ballpark_id,attendance,game_minutes,linescore_v,linescore_h,AB_v,H_v,2B_v,3B_v,HR_v,RBI_v,SH_v,SF_v,HBP_v,BB_v,IBB_v,SO_v,SB_v,CS_v,GIDP_v,CI_v,LOB_v,P_num_v,ERind_v,ERteam_v,WP_v,balk_v,PO_v,ASST_v,ERR_v,PB_v,DP_v,TP_v,AB_h,H_h,2B_h,3B_h,HR_h,RBI_h,SH_h,SF_h,HBP_h,BB_h,IBB_h,SO_h,SB_h,CS_h,GIDP_h,CI_h,LOB_h,P_num_h,ERind_h,ERteam_h,WP_h,balk_h,PO_h,ASST_h,ERR_h,PB_h,DP_h,TP_h,ump_HB_id,ump_HB_name,ump_1B_id,ump_1B_name,ump_2B_id,ump_2B_name,ump_3B_id,ump_3B_name,ump_LF_id,ump_LF_name,ump_RF_id,ump_RF_name,mgr_id_v,mgr_name_v,mgr_id_h,mgr_name_h,pitcher_id_w,pitcher_name_w,pitcher_id_l,pitcher_name_l,pitcher_id_s,pitcher_name_s,GWRBI_id,GWRBI_name,pitcher_start_id_v,pitcher_start_name_v,pitcher_start_id_h,pitcher_start_name_h,batter1_name_v,batter1_id_v,batter1_pos_v,batter2_name_v,batter2_id_v,batter2_pos_v,batter3_name_v,batter3_id_v,batter3_pos_v,batter4_name_v,batter4_id_v,batter4_pos_v,batter5_name_v,batter5_id_v,batter5_pos_v,batter6_name_v,batter6_id_v,batter6_pos_v,batter7_name_v,batter7_id_v,batter7_pos_v,batter8_name_v,batter8_id_v,batter8_pos_v,batter9_name_v,batter9_id_v,batter9_pos_v,batter1_name_h,batter1_id_h,batter1_pos_h,batter2_name_h,batter2_id_h,batter2_pos_h,batter3_name_h,batter3_id_h,batter3_pos_h,batter4_name_h,batter4_id_h,batter4_pos_h,batter5_name_h,batter5_id_h,batter5_pos_h,batter6_name_h,batter6_id_h,batter6_pos_h,batter7_name_h,batter7_id_h,batter7_pos_h,batter8_name_h,batter8_id_h,batter8_pos_h,batter9_name_h,batter9_id_h,batter9_pos_h,misc_info,acqui_info,season,run_diff,home_victory,run_total,date_dblhead,BATAVG_162_h,BATAVG_162_v,OBP_162_h,OBP_162_v,SLG_162_h,SLG_162_v,OPS_162_h,OPS_162_v,SB_162_h,SB_162_v,CS_162_h,CS_162_v,ERR_162_h,ERR_162_v,SO_perc_162_h,SO_perc_162_v,BB_perc_162_h,BB_perc_162_v,BATAVG_30_h,BATAVG_30_v,OBP_30_h,OBP_30_v,SLG_30_h,SLG_30_v,OPS_30_h,OPS_30_v,SB_30_h,SB_30_v,CS_30_h,CS_30_v,ERR_30_h,ERR_30_v,SO_perc_30_h,SO_perc_30_v,BB_perc_30_h,BB_perc_30_v
2352,20090929,1,Tue,MIN,AL,156,DET,AL,156,3,2,60,D,NaN,NaN,NaN,DET05,35243.0,172,0000100002,0100000001,33,11,2,0,0,3,4,2,1,3,1,2,0,0,1,0,10,4,2,2,0,0,30,13,0,0,0,0,36,7,2,0,1,2,1,0,1,2,0,4,0,0,0,0,8,3,3,3,2,0,30,13,0,0,2,0,marsr901,Randy Marsh,hudsm901,Marvin Hudson,emmep901,Paul Emmel,herna901,Angel Hernandez,NaN,(none),NaN,(none),gardr001,Ron Gardenhire,leylj801,Jim Leyland,raucj001,Jon Rauch,lyonb003,Brandon Lyon,nathj001,Joe Nathan,cabro001,Orlando Cabrera,blacn001,Nick Blackburn,porcr001,Rick Porcello,spand001,Denard Span,8,cabro001,Orlando Cabrera,6,mauej001,Joe Mauer,2,kubej002,Jason Kubel,9,cuddm001,Michael Cuddyer,3,yound003,Delmon Young,7,moraj002,Jose Morales,10,tolbm001,Matt Tolbert,5,puntn001,Nick Punto,4,granc001,Curtis Granderson,8,polap001,Placido Polanco,4,thomc002,Clete Thomas,9,cabrm001,Miguel Cabrera,3,huffa001,Aubrey Huff,10,guilc001,Carlos Guillen,7,lairg001,Gerald Laird,2,ingeb001,Brandon Inge,5,santr002,Ramon Santiago,6,NaN,Y,2009,-1,0,5,200909291,0.259596,0.272285,0.325377,0.341213,0.419502,0.424995,0.744879,0.766208,78.0,90.0,34.0,30.0,90.0,69.0,0.205931,0.183504,0.097508,0.104628,0.273166,0.276276,0.344648,0.337912,0.415058,0.417417,0.759705,0.755330,11.0,19.0,5.0,9.0,9.0,14.0,0.195946,0.161161,0.109073,0.093093
1853,20170821,0,Mon,LAN,NL,123,PIT,NL,125,6,5,72,N,NaN,NaN,NaN,PIT08,19094.0,275,000000500001,021000020000,47,10,2,0,2,6,0,0,1,6,0,10,0,0,0,0,12,8,5,5,0,0,36,15,0,0,0,0,46,10,1,0,3,5,0,0,1,4,1,10,0,0,0,0,10,6,6,6,1,0,36,11,1,0,0,0,barrs901,Scott Barry,wolcq901,Quinn Wolcott,onorb901,Brian O'Nora,emmep901,Paul Emmel,NaN,(none),NaN,(none),robed001,Dave Roberts,hurdc001,Clint Hurdle,avill001,Luis Avilan,neved001,Dovydas Neverauskas,strir001,Ross Stripling,puigy001,Yasiel Puig,wooda002,Alex Wood,coleg001,Gerrit Cole,taylc001,Chris Taylor,8,seagc001,Corey Seager,6,turnj001,Justin Turner,5,granc001,Curtis Granderson,7,grany001,Yasmani

In [18]:
df.to_csv('./output_data/input_model_data_01.csv', index=False)